<a href="https://colab.research.google.com/github/greatpak/Breast_cancer_detection/blob/master/Query_Big_Query_from_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-cloud-bigquery

In [10]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [21]:
# Set your Google Cloud Project ID
project_id = 'gtm-mkxz6jtz-ndzln'

In [22]:
print(project_id)

from google.cloud import bigquery
import pandas as pd

# Create a BigQuery client
client = bigquery.Client(project_id)




gtm-mkxz6jtz-ndzln


In [24]:
from google.cloud import bigquery

client = bigquery.Client(project="gtm-mkxz6jtz-ndzln")  # use your real project ID
query_job = client.query("SELECT 1")
print(query_job.result())

query = """
           SELECT
               *
             FROM
          `indigo-codex-393317.matw_international_google_ads.ads_AccountBasicStats_7713128947`;
        """

query_job = client.query(query)

In [25]:
df = query_job.to_dataframe()
df

,customer_id,metrics_clicks,metrics_conversions,metrics_conversions_value,metrics_cost_micros,metrics_impressions,metrics_interaction_event_types,metrics_interactions,metrics_view_through_conversions,segments_ad_network_type,segments_date,segments_device,segments_slot,_LATEST_DATE,_DATA_DATE
0,7713128947,0,0.0,0.0,0,4,[],0,0,SEARCH,2023-08-16,TABLET,SEARCH_TOP,2023-08-24,2023-08-16
1,7713128947,0,0.0,0.0,0,3,[],0,0,SEARCH,2023-08-16,TABLET,SEARCH_OTHER,2023-08-24,2023-08-16
2,7713128947,0,0.0,0.0,0,21,[],0,0,SEARCH_PARTNERS,2023-08-16,TABLET,SEARCH_PARTNER_OTHER,2023-08-24,2023-08-16
3,7713128947,0,0.0,0.0,7245,2,[],0,0,CONTENT,2023-08-16,TABLET,CONTENT,2023-08-24,2023-08-16
4,7713128947,0,0.0,0.0,239950,43,[],0,0,YOUTUBE_SEARCH,2023-08-16,DESKTOP,CONTENT,2023-08-24,2023-08-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,7713128947,0,0.0,0.0,4275516,758,"[""ENGAGEMENT""]",416,0,YOUTUBE_WATCH,2023-08-24,DESKTOP,CONTENT,2023-08-24,2023-08-24
257,7713128947,23,0.0,0.0,14705022,2190,"[""ENGAGEMENT""]",1006,0,YOUTUBE_WATCH,2023-08-24,MOBILE,CONTENT,2023-08-24,2023-08-24
258,7713128947,4,0.0,0.0,600091,364,"[""ENGAGEMENT""]",226,0,YOUTUBE_WATCH,2023-08-24,TABLET,CONTENT,2023-08-24,2023-08-24
259,7713128947,3,0.0,0.0,32639694,17382,"[""ENGAGEMENT""]",13007,0,YOUTUBE_WATCH,2023-08-24,CONNECTED_TV,CONTENT,2023-08-24,2023-08-24


In [ ]:
import json
import psycopg2
from psycopg2.extras import execute_values
from google.cloud import bigquery
import pandas as pd

# ===============================
# 🔹 CONFIG
# ===============================
REDSHIFT_CONFIG = {
    "database": "prod_redshift_db",
    "user": "your_username",
    "password": "your_password",
    "host": "your_redshift_host",
    "port": 5439  # default Redshift port
}

BQ_PROJECT_ID = "gtm-mkxz6jtz-ndzln"

BQ_QUERY = """
    SELECT *
    FROM `indigo-codex-393317.matw_international_google_ads.ads_AccountBasicStats_7713128947`
"""

REDSHIFT_SCHEMA = "bronze"
REDSHIFT_TABLE = "stg_ga_matw_international_google_ads"

# ===============================
# 🔹 Fetch data from BigQuery
# ===============================
client = bigquery.Client(project=BQ_PROJECT_ID)
query_job = client.query(BQ_QUERY)
df = query_job.to_dataframe()

print(f"✅ Pulled {len(df)} rows from BigQuery")


# ===============================
# 🔹 Function: Insert DataFrame into Redshift
# ===============================
def insert_dataframe_to_redshift(df: pd.DataFrame, schema: str, table: str):
    """Insert DataFrame into Redshift using psycopg2 and batch insert"""
    try:
        # Convert dict & list columns into strings
        for column in df.columns:
            if df[column].apply(lambda x: isinstance(x, dict)).any():
                df[column] = df[column].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

            if df[column].apply(lambda x: isinstance(x, list)).any():
                df[column] = df[column].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

        # Replace NaN/None with None (so psycopg2 understands NULL)
        df = df.where(pd.notnull(df), None)

        # Create Redshift connection
        conn = psycopg2.connect(
            dbname=REDSHIFT_CONFIG['database'],
            user=REDSHIFT_CONFIG['user'],
            password=REDSHIFT_CONFIG['password'],
            host=REDSHIFT_CONFIG['host'],
            port=REDSHIFT_CONFIG['port']
        )
        cursor = conn.cursor()

        # Prepare INSERT SQL
        columns = ', '.join(f'"{col}"' for col in df.columns)
        insert_sql = f'INSERT INTO "{schema}"."{table}" ({columns}) VALUES %s'

        # Convert DataFrame to list of tuples
        rows = list(df.itertuples(index=False, name=None))

        # Batch insert
        batch_size = 5000
        for i in range(0, len(rows), batch_size):
            batch = rows[i:i + batch_size]
            execute_values(cursor, insert_sql, batch)
            conn.commit()
            print(f"Inserted {min(i + batch_size, len(rows))}/{len(rows)} rows")

        cursor.close()
        conn.close()
        print(f"✅ Finished inserting {len(rows)} rows into {schema}.{table}")

    except Exception as e:
        print(f"❌ Failed to insert into Redshift: {e}")
        raise


# ===============================
# 🔹 Run Insert
# ===============================
insert_dataframe_to_redshift(df, REDSHIFT_SCHEMA, REDSHIFT_TABLE)